<a href="https://colab.research.google.com/github/saadkarim754/Fake_News_classification/blob/main/fake_news_detection_using_gradient_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

In [9]:
data_fake = pd.read_csv("Fake.csv")
data_true = pd.read_csv("True.csv")

In [ ]:
data_fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [10]:
data_true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


by this stage we have loaded the data

by now our datasets have , 5 coloumns , we add a new column the label i.e 0 or 1.

In [11]:
data_fake["class"]=0
data_true["class"]=1

In [12]:
data_fake.shape,data_true.shape

((23481, 5), (21417, 5))

first number is the rows and second is the column\
now we will remove , the last entries of both csv files for later testing

In [13]:
data_fake_manual_testing = data_fake.tail(10)
for i in range(23480,23470,-1):
    data_fake.drop([i],axis=0,inplace=True)

data_true_manual_testing = data_true.tail(10)
for i in range(21416,21406,-1):
    data_true.drop([i],axis=0,inplace=True)

In [14]:
data_fake.shape,data_true.shape

((23471, 5), (21407, 5))

as u can see 10 rows have been deducted from both csvs.

In [15]:
data_fake_manual_testing["class"]=0
data_true_manual_testing["class"]=1

<ipython-input-15-b0056e2a4ec9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fake_manual_testing["class"]=0
<ipython-input-15-b0056e2a4ec9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_true_manual_testing["class"]=1


In [16]:
data_fake_manual_testing.head(10)

,title,text,subject,date,class
23471,Seven Iranians freed in the prisoner swap have...,"21st Century Wire says This week, the historic...",Middle-east,"January 20, 2016",0
23472,#Hashtag Hell & The Fake Left,By Dady Chery and Gilbert MercierAll writers ...,Middle-east,"January 19, 2016",0
23473,Astroturfing: Journalist Reveals Brainwashing ...,Vic Bishop Waking TimesOur reality is carefull...,Middle-east,"January 19, 2016",0
23474,The New American Century: An Era of Fraud,Paul Craig RobertsIn the last years of the 20t...,Middle-east,"January 19, 2016",0
23475,Hillary Clinton: ‘Israel First’ (and no peace ...,Robert Fantina CounterpunchAlthough the United...,Middle-east,"January 18, 2016",0
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0
23480,10 U.S. Navy Sailors Held by Iranian Military ...,21st Century Wire says As 21WIRE predicted in ...,Middle-east,"January 12, 2016",0


In [17]:
data_true_manual_testing.head(10)

,title,text,subject,date,class
21407,"Mata Pires, owner of embattled Brazil builder ...","SAO PAULO (Reuters) - Cesar Mata Pires, the ow...",worldnews,"August 22, 2017",1
21408,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21409,"U.S., North Korea clash at U.N. arms forum on ...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1
21410,Headless torso could belong to submarine journ...,COPENHAGEN (Reuters) - Danish police said on T...,worldnews,"August 22, 2017",1
21411,North Korea shipments to Syria chemical arms a...,UNITED NATIONS (Reuters) - Two North Korean sh...,worldnews,"August 21, 2017",1
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1
21416,Indonesia to buy $1.14 billion worth of Russia...,JAKARTA (Reuters) - Indonesia will buy 11 Sukh...,worldnews,"August 22, 2017",1


so at this point we have seperated two mini datasets for later **testing**\
now we will create a new dataset , that will have both the true and the fake data sets

In [18]:
data_merge=pd.concat([data_fake,data_true],axis=0)
data_merge.head(10)

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
5,Racist Alabama Cops Brutalize Black Boy While...,The number of cases of cops brutalizing and ki...,News,"December 25, 2017",0
6,"Fresh Off The Golf Course, Trump Lashes Out A...",Donald Trump spent a good portion of his day a...,News,"December 23, 2017",0
7,Trump Said Some INSANELY Racist Stuff Inside ...,In the wake of yet another court decision that...,News,"December 23, 2017",0
8,Former CIA Director Slams Trump Over UN Bully...,Many people have raised the alarm regarding th...,News,"December 22, 2017",0
9,WATCH: Brand-New Pro-Trump Ad Features So Muc...,Just when you might have thought we d get a br...,News,"December 21, 2017",0


In [19]:
data_merge.tail(10)

,title,text,subject,date,class
21397,Germany's Schulz says he would demand U.S. wit...,BERLIN (Reuters) - The leader of Germany s Soc...,worldnews,"August 23, 2017",1
21398,Blunt instrument? What a list of banned articl...,SHANGHAI (Reuters) - An old review of an acade...,worldnews,"August 23, 2017",1
21399,Saudi police release teenager detained for dan...,DUBAI (Reuters) - A 14-year-old boy who was de...,worldnews,"August 22, 2017",1
21400,"The People's Princess, Britons work to keep me...",LONDON (Reuters) - Abdul Daoud spilt most of t...,worldnews,"August 23, 2017",1
21401,"Argentina labor unions protest job losses, Mac...",BUENOS AIRES (Reuters) - Argentina s main labo...,worldnews,"August 22, 2017",1
21402,Exclusive: Trump's Afghan decision may increas...,ON BOARD A U.S. MILITARY AIRCRAFT (Reuters) - ...,worldnews,"August 22, 2017",1
21403,U.S. puts more pressure on Pakistan to help wi...,WASHINGTON (Reuters) - The United States sugge...,worldnews,"August 21, 2017",1
21404,Exclusive: U.S. to withhold up to $290 million...,WASHINGTON (Reuters) - The United States has d...,worldnews,"August 22, 2017",1
21405,Trump talks tough on Pakistan's 'terrorist' ha...,ISLAMABAD (Reuters) - Outlining a new strategy...,worldnews,"August 22, 2017",1
21406,"U.S., North Korea clash at U.N. forum over nuc...",GENEVA (Reuters) - North Korea and the United ...,worldnews,"August 22, 2017",1


now in the next step we remove the columns that are not required

In [20]:
data_merge.columns

Index(['title', 'text', 'subject', 'date', 'class'], dtype='object')

In [21]:
data = data_merge.drop(['title','subject','date'],axis=1)

In [23]:
data.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


as u can see we only have the text and class columns

In [24]:
data.isnull().sum()

,0
text,0
class,0


now we have to randomly shuffle our data

In [25]:
data = data.sample(frac = 1)

In [26]:
data.head(10)

,text,class
18474,Laura Loomer of Rebel Media rushed the stage a...,0
1555,The majority of America has already noticed ...,0
20067,Megyn: What about Tim Allen? You re one of th...,0
16663,This is such a huge opportunity and the feds p...,0
21284,This story is so stupid it hurts Students at W...,0
11638,BEIRUT (Reuters) - Lebanon s interior minister...,1
3052,WASHINGTON (Reuters) - U.S. President Donald T...,1
47,WASHINGTON (Reuters) - U.S. Interior Secretary...,1
6660,The hopes and dreams of millions of bigots fin...,0
3814,While Donald Trump is back peddling on several...,0


as u can see the indexing is not right now we have to fix that


In [27]:
data.reset_index(inplace=True)

In [28]:
data.head(10)

,index,text,class
0,18474,Laura Loomer of Rebel Media rushed the stage a...,0
1,1555,The majority of America has already noticed ...,0
2,20067,Megyn: What about Tim Allen? You re one of th...,0
3,16663,This is such a huge opportunity and the feds p...,0
4,21284,This story is so stupid it hurts Students at W...,0
5,11638,BEIRUT (Reuters) - Lebanon s interior minister...,1
6,3052,WASHINGTON (Reuters) - U.S. President Donald T...,1
7,47,WASHINGTON (Reuters) - U.S. Interior Secretary...,1
8,6660,The hopes and dreams of millions of bigots fin...,0
9,3814,While Donald Trump is back peddling on several...,0


now as u can see we , have reset the index but the previous index
still stays , so now we have to remove that

In [29]:
data.drop(["index"],axis=1,inplace=True)

In [30]:
data.head(10)

,text,class
0,Laura Loomer of Rebel Media rushed the stage a...,0
1,The majority of America has already noticed ...,0
2,Megyn: What about Tim Allen? You re one of th...,0
3,This is such a huge opportunity and the feds p...,0
4,This story is so stupid it hurts Students at W...,0
5,BEIRUT (Reuters) - Lebanon s interior minister...,1
6,WASHINGTON (Reuters) - U.S. President Donald T...,1
7,WASHINGTON (Reuters) - U.S. Interior Secretary...,1
8,The hopes and dreams of millions of bigots fin...,0
9,While Donald Trump is back peddling on several...,0


now weve got a properly formated dataset\
now , , we will pre-process the data set with a function

In [31]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

so we now apply this function on our data variable , on its text column

In [33]:
data['text']=data['text'].apply(wordopt)

In [34]:
data.head(10)

,text,class
0,laura loomer of rebel media rushed the stage a...,0
1,the majority of america has already noticed ...,0
2,megyn what about tim allen you re one of th...,0
3,this is such a huge opportunity and the feds p...,0
4,this story is so stupid it hurts students at w...,0
5,beirut reuters lebanon s interior minister...,1
6,washington reuters u s president donald t...,1
7,washington reuters u s interior secretary...,1
8,the hopes and dreams of millions of bigots fin...,0
9,while donald trump is back peddling on several...,0


as u can see the data is pre-processed now \
now we create the x and y variables

In [35]:
x = data['text']
y = data['class']

now we separate our data for training and testing

In [36]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(x_train)
xv_test = vectorization.transform(x_test)

In [39]:
from sklearn.ensemble import GradientBoostingClassifier
GB = GradientBoostingClassifier(random_state=0)
GB.fit(xv_train, y_train)

GradientBoostingClassifier(random_state=0)

by now we have a fitted model

In [40]:
predict_gb = GB.predict(xv_test)

In [41]:
GB.score(xv_test, y_test)

0.995632798573975

In [42]:
print(classification_report(y_test, predict_gb))

              precision    recall  f1-score   support

           0       1.00      0.99      1.00      5871
           1       0.99      1.00      1.00      5349

    accuracy                           1.00     11220
   macro avg       1.00      1.00      1.00     11220
weighted avg       1.00      1.00      1.00     11220



now we will manual testing of our model

In [43]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"

def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt)
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_GB = GB.predict(new_xv_test)
    return print("\n\nGB Prediction: {} ".format(output_lable(pred_GB[0])))

now the above function will take some news from us, pre-process it and give it to GB.predict function

In [45]:
news = str(input())
manual_testing(news)

A secret government report has revealed that the moon landing in 1969 was staged in a Hollywood studio. The report, leaked by an anonymous whistleblower, claims that all moon landing footage was filmed on Earth and that astronauts never left the planet. Officials have denied the claims, stating they are part of a conspiracy theory with no factual basis.


GB Prediction: Fake News 


In [50]:
news = str(input())
manual_testing(news)


BEIRUT (Reuters) - Lebanon s interior minister said on Thursday that Uber driver suspected of murdering a British embassy worker last week had served time in prison, and he accused the company of not checking criminal records of its drivers. The body of Rebecca Dykes was found strangled on Saturday next to a highway outside Beirut. Police detained a suspect on Monday and said the crime was not politically motivated. Minister Nohad Machnouk said the driver had three priors on his judicial record involving drugs and had been imprisoned on that basis. This company, when it hires drivers, and lets them work within its organisation, does not check their priors , he said at a news conference. An Uber spokesperson said all drivers the company uses in Lebanon are fully licensed by the government and must have a clear judicial record. The spokesperson said a copy of the driver s judicial record published by local media, showing no judgments against the driver, was accurate. Uber confirmed in an